# Compare MendelImpute against Minimac4 and Beagle5 on simulated data

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using SparseArrays
using Plots

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /Users/biona001/.julia/packages/Revise/439di/src/Revise.jl:1108
┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


# Simulate data

### Step 0. Install `msprime`

[msprime download Link](https://msprime.readthedocs.io/en/stable/installation.html).

Some people might need to activate conda environment via `conda config --set auto_activate_base True`. You can turn it off once simulation is done by executing `conda config --set auto_activate_base False`.


### Step 1. Simulate data in terminal

```
python3 msprime_script.py 4000 10000 5000000 2e-8 2e-8 2020 > ./compare5/haplo_ref.vcf
```

Arguments: 
+ Number of haplotypes = 4000
+ Effective population size = 10000 ([source](https://www.the-scientist.com/the-nutshell/ancient-humans-more-diverse-43556))
+ Sequence length = 10 million (same as Beagle 5's choice)
+ Rrecombination rate = 2e-8 (default)
+ mutation rate = 2e-8 (default)
+ seed = 2019

### Step 2: Convert simulated haplotypes to reference haplotypes and target genotype files

+ `haplo_ref.vcf.gz`: haplotype reference files
+ `target.vcf.gz`: complete genotype information
+ `target_masked.vcf.gz`: the same as `target.vcf.gz` except some entries are masked

In [23]:
# import haplotypes and convert to .gz (because beagle throws dumb error when supplied with .vcf)
@time H = convert_ht(Float32, "./compare5/haplo_ref.vcf", as_minorallele=false)

# extract CHROM, POS, ID, REF, ALT info from ref VCF file
vcffile = "compare5/haplo_ref.vcf"
records = nrecords(vcffile)
chrom   = Vector{String}(undef, records)
pos     = zeros(Int, records)
ids     = ["." for i in 1:records]
refs    = Vector{String}(undef, records)
alts    = Vector{String}(undef, records)
reader  = VCF.Reader(openvcf(vcffile, "r"))
for (i, record) in enumerate(reader)
    pos[i]   = VCF.pos(record)
    refs[i]  = VCF.ref(record)
    alts[i]  = VCF.alt(record)[1]
    chrom[i] = VCF.chrom(record)
end
close(reader)

# simulate genotypes from haplotypes and write full genotype data to disk
@time X = simulate_genotypes(H', 500)
make_tgtvcf_file(X, vcffilename="./compare5/target.vcf.gz", marker_chrom=chrom, 
      marker_pos=pos, marker_ID=ids, marker_REF=refs, marker_ALT=alts)

# mask 10% entries
p, n = size(X)
Random.seed!(123)
missingprop = 0.1
X .= ifelse.(rand(Float32, p, n) .< missingprop, missing, X)
masks = ismissing.(X)

# save X to new VCF file
mask_gt("./compare5/target.vcf.gz", masks, des="./compare5/target_masked.vcf.gz")

  7.498143 seconds (141.75 M allocations: 12.415 GiB, 19.45% gc time)
  1.016034 seconds (22.35 k allocations: 331.735 MiB, 16.66% gc time)


# MendelImpute error

In [24]:
# read original target data (all entries observed)
X_complete = convert_gt(Float32, "./compare5/target.vcf.gz"; as_minorallele=false)
Xm = convert_gt(Float32, "./compare5/target_masked.vcf.gz"; as_minorallele=false)
Xm_original = copy(Xm)

500×35232 Array{Union{Missing, Float32},2}:
 0.0       0.0       1.0       0.0       …  0.0       0.0       0.0       0.0
 0.0       0.0        missing  0.0          0.0       0.0       0.0       0.0
  missing  0.0       1.0       0.0          0.0       2.0       2.0       0.0
 0.0       0.0       0.0       0.0           missing  1.0       1.0       0.0
 0.0       0.0       0.0       0.0          0.0       1.0       1.0       0.0
 0.0       0.0        missing  0.0       …  0.0       1.0        missing  0.0
 0.0       0.0       0.0       0.0          0.0       2.0       2.0       0.0
 0.0       0.0       1.0       0.0          0.0        missing  0.0       0.0
 0.0       0.0       1.0       0.0          0.0       0.0       0.0       0.0
 0.0       0.0       2.0       0.0          0.0        missing  0.0       0.0
 0.0       0.0       1.0        missing  …  0.0        missing  0.0       0.0
 0.0       0.0       1.0       0.0          0.0       2.0       2.0       0.0
 0.0       0.0      

In [25]:
# impute
tgtfile = "./compare5/target_masked.vcf.gz"
reffile = "./compare5/haplo_ref.vcf"
outfile = "./compare5/imputed_target.vcf.gz"
width   = 400
@time hs, ph = phase(tgtfile, reffile, impute=true, outfile = outfile, width = width);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile, as_minorallele=false)
n, p = size(X_mendel)
error_rate = sum(X_mendel .!= X_complete) / n / p

 63.048888 seconds (269.70 M allocations: 24.845 GiB, 5.04% gc time)


9.741144414168938e-5

# Beagle 5 error

In [26]:
# run beagle 5 and import imputed data 
run(`java -jar beagle.28Sep18.793.jar gt=./compare5/target_masked.vcf.gz ref=./compare5/haplo_ref.vcf out=./compare5/beagle.result`)

# beagle 5 error rate
X_beagle = convert_gt(Float32, "./compare5/beagle.result.vcf.gz", as_minorallele=false)
n, p = size(X_beagle)
error_rate = sum(X_complete .!= X_beagle) / p / n

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 07:18 PM PST on 22 Jan 2020

Command line: java -Xmx3641m -jar beagle.28Sep18.793.jar
  gt=./compare5/target_masked.vcf.gz
  ref=./compare5/haplo_ref.vcf
  out=./compare5/beagle.result

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,000
Study samples:             500

Window 1 (1:26-4999996)
Reference markers:      35,232
Study markers:          35,232

Burnin  iteration 1:           1 minute 3 seconds
Burnin  iteration 2:           34 seconds
Burnin  iteration 3:           8 seconds
Burnin  iteration 4:           10 seconds
Burnin  iteration 5:           13 seconds
Burnin  iteration 6:           13 seconds

Phasing iteration 1:           8 seconds
Phasing iteration 2:           9 seconds
Phasing iteration 3:           9 seconds
Phasing iteration 4:           8 seconds
Phasing iteration 5:          

1.055858310626703e-5

# Minimac4 error

Need to first convert reference vcf file to m3vcf using minimac3 (on Hoffman)

```Julia
minimac3 = "/u/home/b/biona001/haplotype_comparisons/minimac3/Minimac3/bin/Minimac3"
@time run(`$minimac3 --refHaps haplo_ref.vcf --processReference --prefix haplo_ref`)
```

This took 382.633205 seconds. 

In [36]:
# run minimac 4
minimac4 = "/Users/biona001/Benjamin_Folder/UCLA/research/softwares/Minimac4/build/minimac4"
run(`$minimac4 --refHaps ./compare5/haplo_ref.m3vcf.gz --haps ./compare5/target_masked.vcf.gz --prefix ./compare5/minimac4.result`)

# calculate error
X_minimac = convert_gt(Float32, "./compare5/minimac4.result.dose.vcf.gz", as_minorallele=false)
error_rate = sum(X_complete .!= X_minimac) / n / p



 -------------------------------------------------------------------------------- 
          Minimac4 - Fast Imputation Based on State Space Reduction HMM
 --------------------------------------------------------------------------------
           (c) 2014 - Sayantan Das, Christian Fuchsberger, David Hinds
                             Mary Kate Wing, Goncalo Abecasis 

 Version: 1.0.2;
 Built: Mon Sep 30 11:52:22 PDT 2019 by biona001

 Command Line Options: 
       Reference Haplotypes : --refHaps [./compare5/haplo_ref.m3vcf.gz],
                              --passOnly, --rsid, --referenceEstimates [ON],
                              --mapFile [docs/geneticMapFile.b38.map.txt.gz]
          Target Haplotypes : --haps [./compare5/target_masked.vcf.gz]
          Output Parameters : --prefix [./compare5/minimac4.result],
                              --estimate, --nobgzip, --vcfBuffer [200],
                              --format [GT,DS], --allTypedSites, --meta,
                       

0.02930091961852861